In [25]:
### VERSION 3

import os
from PIL import Image, ImageEnhance, ImageOps
import cv2
import numpy as np
from tqdm import tqdm

class ImageProcessor:
    def __init__(self, input_folder, output_folder):
        """
        Inicializa el procesador de imágenes.
        
        Args:
            input_folder (str): Ruta de la carpeta con imágenes originales
            output_folder (str): Ruta donde se guardarán las imágenes procesadas
        """
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
        
    def create_output_dirs(self):
        """Crea los directorios necesarios para cada versión."""
        os.makedirs(self.output_folder, exist_ok=True)
        for i in range(1, 8):
            os.makedirs(os.path.join(self.output_folder, f"version_{i}"), exist_ok=True)
            os.makedirs(os.path.join(self.output_folder, f"version_{i}_mirror"), exist_ok=True)

    def apply_morning_filter(self, image):
        """
        Filtro de mañana soleada (9:00 AM).
        Partiendo de iluminación de estudio, ajusta para simular luz natural de mañana:
        - Aumenta temperatura de color ligeramente
        - Añade sombras suaves pero definidas
        - Mantiene buena exposición general
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (ligeramente cálida)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Ajuste sutil de canales para luz de mañana
        b, g, r = cv2.split(img_array)
        # Reduce azules ligeramente
        b = cv2.convertScaleAbs(b, alpha=0.95, beta=0)
        # Aumenta rojos suavemente
        r = cv2.convertScaleAbs(r, alpha=1.05, beta=5)
        img_array = cv2.merge([b, g, r])
        
        # Ajuste general de contraste y brillo
        img_array = cv2.convertScaleAbs(img_array, alpha=1.1, beta=5)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajustes finales de imagen
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.15)
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(1.1)
        
        return image

    def apply_late_afternoon_filter(self, image):
        """
        Aplica filtro para simular luz de tarde (5:00 PM).
        Añade tonos cálidos y sombras más pronunciadas.
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (más cálida)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Aumenta los tonos rojos y reduce los azules
        b, g, r = cv2.split(img_array)
        r = cv2.convertScaleAbs(r, alpha=1.15, beta=10)
        b = cv2.convertScaleAbs(b, alpha=0.9, beta=0)
        img_array = cv2.merge([b, g, r])
        
        # Ajusta el contraste general
        img_array = cv2.convertScaleAbs(img_array, alpha=1.1, beta=0)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajusta saturación y contraste
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(1.2)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.15)
        
        return image

    def apply_cloudy_afternoon_filter(self, image):
        """
        Filtro de tarde nublada (5:00 PM).
        Simula luz difusa de día nublado:
        - Reduce el contraste general
        - Enfatiza los tonos medios
        - Añade una sutil dominante fría
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (ligeramente fría)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Ajustes de canales para luz difusa
        b, g, r = cv2.split(img_array)
        # Aumenta azules sutilmente
        b = cv2.convertScaleAbs(b, alpha=1.05, beta=5)
        # Reduce rojos ligeramente
        r = cv2.convertScaleAbs(r, alpha=0.95, beta=0)
        img_array = cv2.merge([b, g, r])
        
        # Reduce contraste general
        img_array = cv2.convertScaleAbs(img_array, alpha=0.9, beta=10)
        
        # Aplica desenfoque sutil para simular dispersión
        img_array = cv2.GaussianBlur(img_array, (3, 3), 0.5)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajustes finales para ambiente nublado
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(0.9)
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(0.85)
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(0.95)
        
        return image

    def apply_night_filter(self, image):
        """
        Filtro nocturno (10:00 PM).
        Simula iluminación artificial nocturna:
        - Reduce significativamente el brillo
        - Añade tono azulado de luz nocturna
        - Aumenta las sombras
        - Simula iluminación artificial
        """
        img_array = np.array(image)
        
        # Ajusta la temperatura de color (fría)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # Ajustes de canales para luz nocturna
        b, g, r = cv2.split(img_array)
        # Enfatiza azules
        b = cv2.convertScaleAbs(b, alpha=1.15, beta=10)
        # Reduce rojos
        r = cv2.convertScaleAbs(r, alpha=0.8, beta=-10)
        # Ajusta verdes
        g = cv2.convertScaleAbs(g, alpha=0.9, beta=-5)
        img_array = cv2.merge([b, g, r])
        
        # Reducción general de luminosidad
        img_array = cv2.convertScaleAbs(img_array, alpha=0.75, beta=-20)
        
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(img_array)
        
        # Ajustes finales para ambiente nocturno
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(0.7)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.3)
        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(0.85)
        
        return image

    def create_mirror_version(self, image):
        """Crea versión espejo de la imagen."""
        return ImageOps.mirror(image)

    def rotate_image(self, image, angle):
        """Rota la imagen por el ángulo especificado."""
        return image.rotate(angle)

    def save_image(self, image, version, filename, is_mirror=False, rotation=0):
        """Guarda la imagen procesada en el directorio correspondiente."""
        subfolder = f"version_{version}_mirror" if is_mirror else f"version_{version}"
        rotation_str = f"_rot{rotation}" if rotation != 0 else ""
        output_filename = f"{filename.split('.')[0]}{rotation_str}.{filename.split('.')[1]}"
        output_path = os.path.join(self.output_folder, subfolder, output_filename)
        image.save(output_path, quality=95, subsampling=0)

    def apply_high_exposure_filter(self, image):
        """Aplica un filtro de muy alta exposición."""
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(2.0)  # Aumenta significativamente el brillo
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(0.8)  # Reduce ligeramente el contraste
        return image

    def apply_low_exposure_filter(self, image):
        """Aplica un filtro de muy baja exposición."""
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(0.4)  # Reduce significativamente el brillo
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.2)  # Aumenta ligeramente el contraste
        return image

    def process_single_image(self, filename):
        """Procesa una única imagen y genera todas sus versiones con rotaciones."""
        input_path = os.path.join(self.input_folder, filename)
        
        try:
            # Abre la imagen original
            original = Image.open(input_path).convert('RGB')
            
            # Lista de funciones de procesamiento para cada versión
            processors = [
                lambda x: x,  # Versión 1: Original
                self.apply_morning_filter,  # Versión 2: Mañana soleada
                self.apply_late_afternoon_filter,  # Versión 3: Tarde
                self.apply_cloudy_afternoon_filter,  # Versión 4: Tarde nublada
                self.apply_night_filter,  # Versión 5: Noche
                self.apply_high_exposure_filter,  # Versión 6: Alta exposición
                self.apply_low_exposure_filter,  # Versión 7: Baja exposición
            ]
            
            # Ángulos de rotación
            rotations = [0, 60, 120, 180, 240, 300]
            
            # Procesa y guarda cada versión con rotaciones
            for version, processor in enumerate(processors, 1):
                # Procesa la imagen
                processed = processor(original.copy())
                
                # Guarda versión normal y espejo con rotaciones
                for rotation in rotations:
                    rotated = self.rotate_image(processed.copy(), rotation)
                    self.save_image(rotated, version, filename, rotation=rotation)
                    
                    mirror_rotated = self.rotate_image(self.create_mirror_version(processed.copy()), rotation)
                    self.save_image(mirror_rotated, version, filename, is_mirror=True, rotation=rotation)
                
        except Exception as e:
            print(f"Error procesando {filename}: {str(e)}")

    # def consolidate_versions(self):
    #     """
    #     Consolida todas las versiones procesadas en una única carpeta dentro del directorio de salida.
    #     Las imágenes se mantienen separadas por cada conjunto procesado.
    #     """
    #     consolidated_folder = os.path.join(self.output_folder, "consolidated")
    #     print(f"\nConsolidando imágenes en: {consolidated_folder}")
        
    #     # Crea el directorio de consolidación
    #     os.makedirs(consolidated_folder, exist_ok=True)
        
    #     # Recorre todas las subcarpetas de versiones
    #     total_copied = 0
    #     for version in range(1, 8):  # Cambiado de 6 a 8 para incluir las nuevas versiones
    #         for subfolder in [f"version_{version}", f"version_{version}_mirror"]:
    #             source_folder = os.path.join(self.output_folder, subfolder)
                
    #             # Verifica si existe la carpeta
    #             if os.path.exists(source_folder):
    #                 # Copia todas las imágenes de la subcarpeta
    #                 for filename in os.listdir(source_folder):
    #                     if any(filename.lower().endswith(ext) for ext in self.valid_extensions):
    #                         source_path = os.path.join(source_folder, filename)
    #                         dest_path = os.path.join(consolidated_folder, filename)
                            
    #                         try:
    #                             # Copia el archivo
    #                             with open(source_path, 'rb') as fsrc:
    #                                 with open(dest_path, 'wb') as fdst:
    #                                     fdst.write(fsrc.read())
    #                             total_copied += 1
    #                         except Exception as e:
    #                             print(f"Error copiando {filename}: {str(e)}")
        
    #     print(f"Se consolidaron {total_copied} imágenes exitosamente en {consolidated_folder}")
    
    def consolidate_versions(self):
        """
        Consolida todas las versiones procesadas en una única carpeta dentro del directorio de salida.
        Las imágenes se mantienen separadas por cada conjunto procesado.
        """
        consolidated_folder = os.path.join(self.output_folder, "consolidated")
        print(f"\nConsolidando imágenes en: {consolidated_folder}")
        
        # Crea el directorio de consolidación
        os.makedirs(consolidated_folder, exist_ok=True)
        
        # Recorre todas las subcarpetas de versiones
        total_copied = 0
        for version in range(1, 8):  # Incluye todas las versiones del 1 al 7
            for subfolder in [f"version_{version}", f"version_{version}_mirror"]:
                source_folder = os.path.join(self.output_folder, subfolder)
                
                # Verifica si existe la carpeta
                if os.path.exists(source_folder):
                    # Copia todas las imágenes de la subcarpeta
                    for filename in os.listdir(source_folder):
                        if any(filename.lower().endswith(ext) for ext in self.valid_extensions):
                            source_path = os.path.join(source_folder, filename)
                            # Añade el prefijo de la versión al nombre del archivo para evitar sobrescrituras
                            new_filename = f"{subfolder}_{filename}"
                            dest_path = os.path.join(consolidated_folder, new_filename)
                            
                            try:
                                # Copia el archivo
                                with open(source_path, 'rb') as fsrc:
                                    with open(dest_path, 'wb') as fdst:
                                        fdst.write(fsrc.read())
                                total_copied += 1
                            except Exception as e:
                                print(f"Error copiando {filename}: {str(e)}")
        
        print(f"Se consolidaron {total_copied} imágenes exitosamente en {consolidated_folder}")
    def process_all_images(self):
        """Procesa todas las imágenes en el directorio de entrada."""
        # Crea directorios de salida
        self.create_output_dirs()
        
        # Obtiene lista de imágenes
        images = [f for f in os.listdir(self.input_folder) 
                 if os.path.splitext(f)[1].lower() in self.valid_extensions]
        
        if not images:
            print("No se encontraron imágenes para procesar.")
            return
        
        print(f"Procesando {len(images)} imágenes...")
        
        # Procesa cada imagen con barra de progreso
        for filename in tqdm(images):
            self.process_single_image(filename)
            
        print("\n¡Procesamiento completado!")

def main():
    # input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/test/"
    # output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/test/"
    # processor = ImageProcessor(input_folder, output_folder)
    # processor.process_all_images()
    # processor.consolidate_versions() 
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/ford_fiesta_zetec/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/ford_fiesta_zetec/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    processor.consolidate_versions() 
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/mazda_iconic_sp/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mazda_iconic_sp/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    processor.consolidate_versions() 
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/mini_cooper_d/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mini_cooper_d/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    processor.consolidate_versions() 
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/rover_p6_3500s/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/rover_p6_3500s/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    processor.consolidate_versions()
    
    input_folder = "C:/git/IAClass/16_projectU3_cnn_cars/captures/volkswagen_up/"
    output_folder = "C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/volkswagen_up/"
    processor = ImageProcessor(input_folder, output_folder)
    processor.process_all_images()
    processor.consolidate_versions()

if __name__ == "__main__":
    main()

Procesando 166 imágenes...


100%|██████████| 166/166 [00:13<00:00, 12.56it/s]



¡Procesamiento completado!

Consolidando imágenes en: C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/ford_fiesta_zetec/consolidated
Se consolidaron 13944 imágenes exitosamente en C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/ford_fiesta_zetec/consolidated
Procesando 167 imágenes...


100%|██████████| 167/167 [00:12<00:00, 13.24it/s]



¡Procesamiento completado!

Consolidando imágenes en: C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mazda_iconic_sp/consolidated
Se consolidaron 14028 imágenes exitosamente en C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mazda_iconic_sp/consolidated
Procesando 162 imágenes...


...


¡Procesamiento completado!

Consolidando imágenes en: C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mini_cooper_d/consolidated
Se consolidaron 13608 imágenes exitosamente en C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/mini_cooper_d/consolidated
Procesando 158 imágenes...


100%|██████████| 158/158 [00:13<00:00, 12.00it/s]



¡Procesamiento completado!

Consolidando imágenes en: C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/rover_p6_3500s/consolidated
Se consolidaron 13272 imágenes exitosamente en C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/rover_p6_3500s/consolidated
Procesando 168 imágenes...


100%|██████████| 168/168 [00:14<00:00, 11.45it/s]



¡Procesamiento completado!

Consolidando imágenes en: C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/volkswagen_up/consolidated
Se consolidaron 14112 imágenes exitosamente en C:/git/IAClass/16_projectU3_cnn_cars/prossesed_image/volkswagen_up/consolidated
